In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "ai4bharat/IndicCorpV2",
    name="indiccorp_v2",
    split="hin_Deva",
    streaming=True  # Optional: avoid full download
)

print(next(iter(dataset)))

README.md: 0.00B [00:00, ?B/s]

{'text': 'लोगों को बिलों संबंधी सुविधा देना ही उनका काम'}


In [ ]:
def custom_sentence_tokenizer(text):
    # Split by Hindi sentence enders: । ? !
    # Keep punctuation attached to sentence
    sentences = re.split(r'(?<=[।!,?])\s+', text.strip())
    return [s.strip() for s in sentences if s.strip()]

In [ ]:
import re

def custom_word_tokenizer(text):
    pattern = (
        r'\b[\w\.-]+@[\w\.-]+\.\w+\b'       # emails
        r'|https?://\S+'                    # URLs
        r'|\d+\.\d+|\d+'                    # numbers
        r'|[\u0900-\u097F]+'                # Hindi words (matras included)
        r'|[^\s\w\u0900-\u097F]'            # punctuation
    )
    return re.findall(pattern, text)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ai4bharat/IndicCorpV2", name="indiccorp_v2", split="hin_Deva", streaming=True)

total_sentences = 0
total_words = 0
total_chars = 0
unique_words = set()

MAX_PARAGRAPHS = 500_000  # Optional limit for speed

for i, example in enumerate(dataset):
    text = example["text"]

    # Custom sentence tokenizer
    sentences = custom_sentence_tokenizer(text)
    for sent in sentences:
        total_sentences += 1
        words = custom_word_tokenizer(sent)
        total_words += len(words)
        total_chars += sum(len(word) for word in words)
        unique_words.update(words)

    if i + 1 >= MAX_PARAGRAPHS:
        break

# Final statistics
print(f"Total sentences: {total_sentences}")
print(f"Total words: {total_words}")
print(f"Total characters: {total_chars}")
print(f"Average sentence length (words/sentence): {total_words / total_sentences:.2f}")
print(f"Average word length (characters/word): {total_chars / total_words:.2f}")
print(f"Type/Token Ratio: {len(unique_words) / total_words:.4f}")

Total sentences: 1020630
Total words: 15331056
Total characters: 58900837
Average sentence length (words/sentence): 15.02
Average word length (characters/word): 3.84
Type/Token Ratio: 0.0160


In [ ]:
import itertools

# Take first 3 paragraphs from the streaming dataset
for i, example in enumerate(itertools.islice(dataset, 3), start=1):
    text = example["text"].strip()
    print(f"\n=== Row {i} Original Text ===")
    print(text)

    # Sentence Tokenization
    sentences = custom_sentence_tokenizer(text)
    print("\n--- Sentences ---")
    for idx, sent in enumerate(sentences, 1):
        print(f"{idx}. {sent}")

    # Word Tokenization for the first sentence
    if sentences:
        print("\n--- Words in Sentence 1 ---")
        words = custom_word_tokenizer(sentences[0])
        print(words)
    else:
        print("\n(No valid sentences found in this row)")


=== Row 1 Original Text ===
लोगों को बिलों संबंधी सुविधा देना ही उनका काम

--- Sentences ---
1. लोगों को बिलों संबंधी सुविधा देना ही उनका काम

--- Words in Sentence 1 ---
['लोगों', 'को', 'बिलों', 'संबंधी', 'सुविधा', 'देना', 'ही', 'उनका', 'काम']

=== Row 2 Original Text ===


--- Sentences ---

(No valid sentences found in this row)

=== Row 3 Original Text ===
इनेलो 1987 में उस वक्त ऐसे ही दोराहे पर खड़ी थी, जब पूर्व उपप्रधानमंत्री देवीलाल ने अपने पुत्र ओमप्रकाश चौटाला को अपना राजनीतिक उत्तराधिकारी घोषित किया था। हालांकि तब पार्टी पर देवीलाल की मजबूत पकड़ के चलते पार्टी टूटने से बच गई थी। 1989 में देवीलाल केन्द्र की राजनीति में सक्रिय हो गए थे और उनके उपप्रधानमंत्री बनने के पश्चात् उनके तीन बेटों जगदीश सिंह, रणजीत सिंह और ओमप्रकाश चौटाला में से रणजीत और ओमप्रकाश के बीच हरियाणा में उनकी राजनीतिक विरासत को लेकर जंग शुरू हो गई थी। उन परिस्थितियों में देवीलाल ने कड़ा निर्णय लेते हुए पार्टी की बागडोर ओमप्रकाश चौटाला के हवाले कर दी थी, जिसके बाद रणजीत की बगावत का असर पार्टी, संगठन और उनकी स

In [11]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define path to save tokenized sentences in your Google Drive
output_file = "/content/drive/MyDrive/tokenized_sentences.txt"

# Create the directory if it doesn't exist
import os
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Initialize counters
total_sentences = 0
total_words = 0
total_chars = 0
unique_words = set()
MAX_PARAGRAPHS = 500000  # Process first 5 lakh paragraphs

# Save tokenized sentences directly into Google Drive
with open(output_file, "w", encoding="utf-8") as f:
    for i, example in enumerate(dataset):
        text = example["text"].strip()
        sentences = custom_sentence_tokenizer(text)

        for sent in sentences:
            words = custom_word_tokenizer(sent)
            if not words:
                continue

            tokenized_sentence = " ".join(words)
            f.write(tokenized_sentence + "\n")

            total_sentences += 1
            total_words += len(words)
            total_chars += sum(len(word) for word in words)
            unique_words.update(words)

        if i + 1 >= MAX_PARAGRAPHS:  # Stop after 5 lakh paragraphs
            break

print(f"✅ Tokenized sentences saved to: {output_file}")
print(f"📄 Total Sentences: {total_sentences}")
print(f"🔤 Total Words: {total_words}")
print(f"🔠 Unique Words: {len(unique_words)}")

Mounted at /content/drive
✅ Tokenized sentences saved to: /content/drive/MyDrive/tokenized_sentences.txt
📄 Total Sentences: 1020545
🔤 Total Words: 15331056
🔠 Unique Words: 245959


In [12]:
print(f"Total sentences: {total_sentences}")
print(f"Total words: {total_words}")
print(f"Total characters: {total_chars}")
print(f"Average sentence length: {total_words / total_sentences:.2f} words")
print(f"Average word length: {total_chars / total_words:.2f} characters")
print(f"Type/Token Ratio: {len(unique_words) / total_words:.4f}")

Total sentences: 1020545
Total words: 15331056
Total characters: 58900837
Average sentence length: 15.02 words
Average word length: 3.84 characters
Type/Token Ratio: 0.0160
